In [ ]:
# How to use LLMs on Vector cluster
# 1) KScope for inference: https://kscope.vectorinstitute.ai/
# 2) VectorInference for inference and logit extraction:
    # https://github.com/VectorInstitute/vector-inference/blob/main/examples/logits/logits.py

In [3]:
import openai
from time import sleep
import os

In [7]:
# 1) Load the environment variables
from dotenv import load_dotenv

load_dotenv() 

True

In [60]:
# Specify model name (you can change the model easily).
model="Meta-Llama-3.1-8B-Instruct"
# model="Llama-2-7b-chat-hf"

In [61]:
messages = [
    {
        "role": "user",
        "content": "Please introduce yourself.",
    }
]

client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# If model is not yet available, try again after some delay.
output = None
while output is None:
    try:
        output = client.chat.completions.create(
            model=model,
            messages=messages,
        )
    
    except openai.APIError as e:
        print(e)
        sleep(10)

print(output.choices[0].message.content)

I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."


In [10]:
def construct_input(question):
    
    prompt = [
        {
            "role": "user",
            "content": question,
        }
    ]
    
    return prompt

In [11]:
small_generation_config = {"max_tokens": 20, "temperature": 0.9}
moderate_generation_config = {"max_tokens": 200, "temperature": 0.9}

Simple Example

In [12]:
# message
prompt = construct_input("What is the capital of Canada?")
generation = client.chat.completions.create(model=model, messages = prompt, **small_generation_config)
# Extract the text from the returned generation
print(generation.choices[0].message.content)

The capital of Canada is Ottawa. It is located in the province of Ontario and is home to many


# Few-Shot Chain of Thought Prompting

We'll start by prompting the model to solve some word problems and build up to using the Few-Shot CoT method proposed in "Chain-of-Thought Prompting Elicits Reasoning in Large Language Models"

First try "zero-shot prompting".

In [13]:
zero_shot_prompt = (
    "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have? just give final answer with no explanation."
)
zero_shot_prompt = construct_input(zero_shot_prompt)

generation_example = client.chat.completions.create(model=model,messages = zero_shot_prompt,  **small_generation_config)
print(generation_example.choices[0].message.content)

31


The correct answer is 9.

Now let's try standard few-shot prompt.

In [14]:
few_shot_prompt = (
    "Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis "
    "balls does he have now?\nA: The answer is 11.\n\nQ: Benjamin is taking bottle inventory. He has two cases with "
    "15 bottles in each and one with 7. How many bottles are there in total?\nA: The answer is 37.\n\nQ: The "
    "cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?\nA: "
    "The answer is ...\nJust give the final answer to the last question with no explanation."
)
few_shot_message = construct_input(few_shot_prompt)

print(few_shot_prompt)

Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
A: The answer is 11.

Q: Benjamin is taking bottle inventory. He has two cases with 15 bottles in each and one with 7. How many bottles are there in total?
A: The answer is 37.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
A: The answer is ...
Just give the final answer to the last question with no explanation.


In [15]:
generation_example = client.chat.completions.create(model=model,messages = few_shot_message,  **small_generation_config)
print(generation_example.choices[0].message.content)

25


Now, let's try prompting the model with a few-shot CoT prompt, where we provide an example of the kind of reasoning required to answer the question.

In [16]:
few_shot_cot_prompt = (
    "Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis "
    "balls does he have now?\nA: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. "
    "5 + 6 = 11. The answer is 11.\n\nQ: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 "
    "more, how many apples do they have?\nA:"
)

few_shot_cot_prompt_message = construct_input(few_shot_cot_prompt)
print(few_shot_cot_prompt)

Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
A: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
A:


In [17]:
generation_example = client.chat.completions.create(model=model,messages = few_shot_cot_prompt_message, **moderate_generation_config)
print(generation_example.choices[0].message.content)

To find the answer, subtract the apples used from the initial number (23 - 20 = 3) and then add the apples bought (3 + 6 = 9).

The final answer is 9.


## An example from the AQuA: Algebraic Word Problems task.

Let's try to compare few-shot prompting with few-shot CoT for slightly different kind of problem. This example is drawn from the AQuA: Algebraic Word Problems task.

In [18]:
few_shot_prompt = (
    "Q: John found that the average of 15 numbers is 40. If 10 is added to each number then the mean of "
    "the numbers is? Answer Choices: (a) 50 (b) 45 (c) 65 (d) 78 (e) 64\nA: The answer is (a).\n\nQ: The capacity of "
    "a tank of dimensions (8 m × 6 m × 2.5 m) is Answer Choices: (a) 120 litres (b) 1200 litres (c) 12000 litres (d) "
    "120000 litres (e) None of these\nA: \n what is the answer to the second Q with a,b,c,d,or e and don't say a single word except the final answer."
)
few_shot_prompt_message = construct_input(few_shot_prompt)
print(few_shot_prompt)

Q: John found that the average of 15 numbers is 40. If 10 is added to each number then the mean of the numbers is? Answer Choices: (a) 50 (b) 45 (c) 65 (d) 78 (e) 64
A: The answer is (a).

Q: The capacity of a tank of dimensions (8 m × 6 m × 2.5 m) is Answer Choices: (a) 120 litres (b) 1200 litres (c) 12000 litres (d) 120000 litres (e) None of these
A: 
 what is the answer to the second Q with a,b,c,d,or e and don't say a single word except the final answer.


In [19]:
generation_example = client.chat.completions.create(model=model,messages = few_shot_prompt_message, **small_generation_config)
print(generation_example.choices[0].message.content)

(e)


The correct choice for this problem is "d".

In [20]:
few_shot_cot_prompt = (
    "Q: John found that the average of 15 numbers is 40. If 10 is added to each number then the mean of the numbers "
    "is? Answer Choices: (a) 50 (b) 45 (c) 65 (d) 78 (e) 64\nA: If 10 is added to each number, then the mean of the "
    "numbers also increases by 10. So the new mean would be 50. The answer is (a).\n\nQ: The capacity of "
    "a tank of dimensions (8 m × 6 m × 2.5 m) is Answer Choices: (a) 120 litres (b) 1200 litres (c) 12000 litres (d) "
    "120000 litres (e) None of these \nA:"
)
few_shot_cot_prompt_message = construct_input(few_shot_cot_prompt)
print(few_shot_cot_prompt)

Q: John found that the average of 15 numbers is 40. If 10 is added to each number then the mean of the numbers is? Answer Choices: (a) 50 (b) 45 (c) 65 (d) 78 (e) 64
A: If 10 is added to each number, then the mean of the numbers also increases by 10. So the new mean would be 50. The answer is (a).

Q: The capacity of a tank of dimensions (8 m × 6 m × 2.5 m) is Answer Choices: (a) 120 litres (b) 1200 litres (c) 12000 litres (d) 120000 litres (e) None of these 
A:


In [21]:
generation_example = client.chat.completions.create(model=model,messages = few_shot_cot_prompt_message, **moderate_generation_config)
print(generation_example.choices[0].message.content)

## Step 1: Calculate the volume of the tank in cubic meters.
The volume of a rectangular tank can be calculated by multiplying its length, width, and height. In this case, the volume is given by V = length × width × height = 8 m × 6 m × 2.5 m = 120 cubic meters.

## Step 2: Convert the volume from cubic meters to liters.
Since 1 cubic meter is equal to 1000 liters, we can convert the volume of the tank by multiplying it by 1000: 120 cubic meters × 1000 liters/cubic meter = 120,000 liters.

## Step 3: Identify the correct answer choice that matches the calculated volume of the tank.
The calculated volume of the tank is 120,000 liters, which matches answer choice (d).

The final answer is: $\boxed{120000}$


Sometimes the examples are not good enough.

# Zero-Shot Chain of Thought Prompting

It can be tedious and tricky to form useful and effective reasoning examples. Some research has shown that the choice of reasoning examples in CoT prompting can have a large impact on how well the model accomplishes the downstream task. So let's try a zero-shot CoT approach devised in "Large Language Models are Zero-Shot Reasoners"

In [26]:
few_shot_prompt = (
    "Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis "
    "balls does he have now?\nA: The answer is 11.\n\nQ: There are 64 students trying out for the school's trivia "
    "teams. If 36 of them didn't get picked for the team and the rest were put into 4 groups, how many students would "
    "be in each group?\nA: \nJust give the final answer to the last question with no explanation."
)


few_shot_prompt_message = construct_input(few_shot_prompt)
print(few_shot_prompt)

Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
A: The answer is 11.

Q: There are 64 students trying out for the school's trivia teams. If 36 of them didn't get picked for the team and the rest were put into 4 groups, how many students would be in each group?
A: 
Just give the final answer to the last question with no explanation.


In [29]:
generation_example = client.chat.completions.create(model=model,messages = few_shot_prompt_message, **small_generation_config)
print(generation_example.choices[0].message.content)

23.


The correct answer to this problem is 7.

Could you get the correct answer with this example?


# TASK: 

Try to do CoT without adding examples.


Split into two stages:

1) Reasoning Generation   
2) Answer Extraction

In [72]:
question_prompt = ("Q: There are 64 students trying out for the school's trivia teams."
                   "If 36 of them didn't get picked for the team and the rest were put into 4 groups, how many"
                   "students would be in each group?\nA: \nJust give the final asnwer." )

question_prompt_message = construct_input(question_prompt)
print(question_prompt)

Q: There are 64 students trying out for the school's trivia teams.If 36 of them didn't get picked for the team and the rest were put into 4 groups, how manystudents would be in each group?
A: 
Just give the final asnwer.


In [73]:
generation_example = client.chat.completions.create(model=model, messages = question_prompt_message, **small_generation_config)
reasoning_generation = generation_example.choices[0].message.content
print(reasoning_generation)

9


Try to get the correct answer (7) with no example.

In [18]:
# Hint: break down the problem into two steps. First, ask the model for reasoning,
# then, given the reasoning, ask for the final answer by appending "\nTherefore, the answer is" followed by the reasoning.

In [ ]:
# reasoning_prompt = ...